## Scraping Rottentomatoes: https://www.rottentomatoes.com/browse/movies_in_theaters/

In [1]:
import requests
import random
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import duckdb
from urllib import parse
from urllib.parse import urljoin
import functions_project 

In [2]:
#create fake user agents to use while requesting
user_agents_list = [
    'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
    ]

### TRIAL AND ERROR BLOCK (last part to fix!)

In [29]:
### TRIAL AND ERROR BLOCK
## This block needs improvement. Strangely enough when the page id > 5, the url is not opening
# so we can not scrape the next pages?? It is aways going back to /?=page5. 
# Although when you click on the load more button you see the page id is going up and so my program does

# Get request to an example page url

##till /?page=5 it works correctly
# url = 'https://www.rottentomatoes.com/browse/movies_at_home/?page=2' 

##for example this one not. You get the same respons of page id 5, while using another page id > 5
url = "https://www.rottentomatoes.com/browse/movies_at_home/?page=12"

r = requests.get(url, headers={'User-Agent': random.choice(user_agents_list)})
# print(r.text)
soup = BeautifulSoup(r.content, "html.parser")

pages = soup.find_all('a', attrs = {'data-qa':'discovery-media-list-item-caption'}) 
for p in pages:
        link_id = p.get('href')
        m_link = urljoin(url, link_id)
        print(m_link)

https://www.rottentomatoes.com/m/cocaine_bear
https://www.rottentomatoes.com/m/operation_fortune_ruse_de_guerre
https://www.rottentomatoes.com/m/children_of_the_corn
https://www.rottentomatoes.com/m/wildflower_2022
https://www.rottentomatoes.com/m/magic_mikes_last_dance
https://www.rottentomatoes.com/m/followers_2021
https://www.rottentomatoes.com/m/last_sentinel_2023
https://www.rottentomatoes.com/m/88_2022
https://www.rottentomatoes.com/m/infinite_sea
https://www.rottentomatoes.com/m/reggie_2023
https://www.rottentomatoes.com/m/jack_2021
https://www.rottentomatoes.com/m/everything_everywhere_all_at_once
https://www.rottentomatoes.com/m/the_whale_2022
https://www.rottentomatoes.com/m/boston_strangler_2023
https://www.rottentomatoes.com/m/luther_the_fallen_sun
https://www.rottentomatoes.com/m/shazam
https://www.rottentomatoes.com/m/rrr
https://www.rottentomatoes.com/m/dragged_across_concrete
https://www.rottentomatoes.com/m/the_banshees_of_inisherin
https://www.rottentomatoes.com/m/mis

### Get pagination

In [19]:
# Get all pages id /urls (after click load more button)
##(so I get all the urls with the correct pageids here correctly)
urls = ['https://www.rottentomatoes.com/browse/movies_at_home/']
url = 'https://www.rottentomatoes.com/browse/movies_at_home/'
for count in range(2,7,1):                    ###### set the number of page ids in the range (now 7)
    count = '?page=' + str(count)
    url = urljoin(url, count)
    urls.append(url)
# urls

## Scraping all pages

In [34]:
### PUT IT ALL TOGETHER!!!!!
rot_dict = dict()
rtom_dict = dict()
list_with_dicts = []
lab_list = []
title = []
val = []
movie_pages = []


for url in urls:
#     print(url)

    ## Get request to url
    r = requests.get(url, headers={'User-Agent': random.choice(user_agents_list)})
    soup = BeautifulSoup(r.content, "html.parser")

    ## Get the movie page ids from the current url
    # Get ids of the movie pages (after click on movie) on the current page id
    pages = soup.find_all('a', attrs = {'data-qa':'discovery-media-list-item-caption'}) 
    for p in pages:
        link_id = p.get('href')
        m_link = urljoin(url, link_id)
        movie_pages.append(m_link)
    
    # Request movie page from url
    for u in movie_pages:
        req = requests.get(u, headers={'User-Agent': random.choice(user_agents_list)})
        soup = BeautifulSoup(req.content, "html.parser")
        
        ## Get movie information (title and values)
        #Get title, labels and values
        title_of_movie = soup.find("h1", {'class':"scoreboard__title"})
        if title_of_movie != None:
            title.append(title_of_movie.text)
        else:
            continue

        labels = soup.find_all('b', attrs = {'data-qa': 'movie-info-item-label'})
        values = soup.find_all('span', attrs = {'data-qa': 'movie-info-item-value'})
            
        # Get values (Genre)
        for v in values:
            val.append(v.text.strip())
        
        # append values to dict with title as Key and genre (or other) as Value
        rot_dict['title'] = title[0]
        rot_dict['genre1'] = val[0] #on some moviepages genre is in [0], sometimes in [1]
        rot_dict['genre2'] = val[1]
#         rot_dict['Original Language'] = val[2]
#         rot_dict['Director'] = val[3]
#         rot_dict['Producer'] = val[4]
#         rot_dict['Release Date (Theaters)'] = val[5]
#         rot_dict['Release Date (Streaming)'] = val[6]
#         rot_dict['Runtime'] = val[7]
#         rot_dict['Distributor'] = val[8]

        # After adding the imformation to the dictionary, delete the information in the lists
        #to start over again in the next iteration
        val = []
        title = []
        lab_list= []
        movie_pages = []
    
        #### FINAL list of dicts with all the scraped information
        # Append the dictionary with the information of the movie to a list 
        # The result is a list with Dictionaries with every dictionary being the information of one movie
        list_with_dicts.append(rot_dict)

        # Emptying dict to start over again in the new iteration
        rot_dict = dict()


In [35]:
# list_with_dicts

In [36]:
df = pd.DataFrame.from_dict(list_with_dicts)
df

,title,genre1,genre2
0,Cocaine Bear,R (Language Throughout|Drug Content|Bloody Vio...,"Mystery & thriller, \n ..."
1,Operation Fortune: Ruse de guerre,R (Language and Violence),"Action, \n \n ..."
2,Children of the Corn,R (Violence and Bloody Images),Horror
3,Wildflower,R (A Sexual Reference|Teen Drinking|Some Langu...,"Comedy, \n \n ..."
4,Magic Mike's Last Dance,R (Sexual Material and Language),"Comedy, \n \n ..."
...,...,...,...
436,Three Thousand Years of Longing,R (Graphic Nudity|Brief Violence|Some Sexual C...,"Fantasy, \n \n ..."
437,The Gray Man,PG-13 (Strong Violence|Strong Language),"Action, \n \n ..."
438,Get Out,R (Language|Bloody Images|Sexual References|Vi...,"Horror, \n \n ..."
439,Marvel's the Avengers,PG-13 (Intense Sci-Fi Action/Violence|A Mild D...,"Action, \n \n ..."


### Clean string values in each column 

In [37]:
functions_project.map_cleaned_strings(df,'title')
functions_project.map_cleaned_strings(df,'genre1')
functions_project.map_cleaned_strings(df,'genre2')

df

,title,genre1,genre2
0,cocaine bear,r language throughoutdrug contentbloody violen...,mystery thriller \n ...
1,operation fortune ruse de guerre,r language and violence,action \n \n ...
2,children of the corn,r violence and bloody images,horror
3,wildflower,r a sexual referenceteen drinkingsome language,comedy \n \n ...
4,magic mikes last dance,r sexual material and language,comedy \n \n ...
...,...,...,...
436,three thousand years of longing,r graphic nuditybrief violencesome sexual content,fantasy \n \n ...
437,the gray man,pg13 strong violencestrong language,action \n \n ...
438,get out,r languagebloody imagessexual referencesviolence,horror \n \n ...
439,marvels the avengers,pg13 intense scifi actionviolencea mild drug r...,action \n \n ...


In [40]:
df['genre2'][1].strip

<function str.strip(chars=None, /)>

In [46]:
df['genre2'] = df['genre2'].str.replace("\n","")
df['genre2'] = df['genre2'].str.strip()

df

,title,genre1,genre2
0,cocaine bear,r language throughoutdrug contentbloody violen...,mystery thriller ...
1,operation fortune ruse de guerre,r language and violence,action ...
2,children of the corn,r violence and bloody images,horror
3,wildflower,r a sexual referenceteen drinkingsome language,comedy ...
4,magic mikes last dance,r sexual material and language,comedy ...
...,...,...,...
436,three thousand years of longing,r graphic nuditybrief violencesome sexual content,fantasy ...
437,the gray man,pg13 strong violencestrong language,action ...
438,get out,r languagebloody imagessexual referencesviolence,horror ...
439,marvels the avengers,pg13 intense scifi actionviolencea mild drug r...,action ...


### Add as table to database

In [ ]:
##to do
# # create a connection to a file called 'imdb_reviews.db'
# con = duckdb.connect('imdb_reviews.db')

# # create a table and load data into it
# con.sql("CREATE TABLE IF NOT EXISTS budget_data AS SELECT * FROM df")

# con.close()